In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [26]:
class Card:
    def __init__(self, cost=0, victory_points=0, card_type=None, money=0):
        self.cost = cost
        self.victory_points = victory_points
        self.card_type = card_type
        self.money = money

In [212]:
cl = {
        "copper": Card(card_type="treasure", money=1),
        "silver": Card(card_type="treasure", cost=3, money=2),
        "gold": Card(card_type="treasure", cost=6, money=3),
        "estate": Card(card_type="victory", cost=2, victory_points=1),
        "duchy": Card(card_type="victory", cost=5, victory_points=3),
        "province": Card(card_type="victory", cost=8, victory_points=6),
     }

cards = list(cl.keys())

In [507]:
empty_deck = pd.DataFrame(np.array([[0, 0, 0, 0, 0, 0]]), columns = cards)
starting_deck = pd.DataFrame(np.array([[7, 0, 0, 3, 0, 0]]), columns = cards)
supply_pile = pd.DataFrame(np.array([[50, 30, 20, 5, 5, 5]]), columns = cards)

In [436]:
class Player:
    def __init__(self, index):
        self.index = index
        self.all_cards = starting_deck.copy()
        self.deck = []  # must be ordered
        self.discarded = starting_deck.copy()
        self.hand = empty_deck.copy()
        self.played = empty_deck.copy()
        
        self.actions = 1
        self.buys = 1
        self.money_in_play = 0
        self.score = 0
        
        self.shuffle()
        self.draw(5)
        
    def shuffle(self):  #called when player needs to draw but the deck is empty
        self.deck = []  # should be redundant but just in case
        for card in self.discarded:
            for _ in range(self.discarded[card].values[0]):
                self.deck.append(card)
            self.discarded[card] = 0
                
        np.random.shuffle(self.deck)
        
    def draw(self, n=1):                
        for _ in range(n):
            if not self.deck:
                self.shuffle()
                if not self.deck:  # no cards in deck or discard pile
                    break
            self.hand[self.deck[0]] += 1
            self.deck = self.deck[1:]
    
    def discard_hand(self):
        for card in self.hand:
            a = self.hand[card].values[0]
            self.hand[card] -= a
            self.discarded[card] += a
            
    
    def discard_played(self):
        for card in self.played:
            a = self.played[card].values[0]
            self.played[card] -= a
            self.discarded[card] += a
                  
    
    def play(self, card, n=1):
        if self.hand[card].values[0] >= n:
            self.hand[card] -= n
            self.played[card] += n
            
    
    def gain(self, card):
        self.discarded[card] += 1
        self.all_cards[card] += 1
        
            
    def end_turn(self):
        self.money_in_play = 0
        self.discard_hand()
        self.discard_played()
        self.draw(5)
        self.actions = 1
        self.buys = 1

In [438]:
class GameState:
    def __init__(self):
        self.num_players = 2
        self.players = [Player(i) for i in range(self.num_players)]
        self.active_player_index = 0
        self.active_player = self.players[0]
        
        self.supply_pile = supply_pile.copy()
        
        
    def play_all_treasures(self):
        h = self.active_player.hand
        for card in h:
            if cl[card].card_type == 'treasure':
                amount = h[card].values[0]    # how many treasures are in hand
                self.active_player.play(card, n=amount)  # move them out of hand and into play
                self.active_player.money_in_play += amount*cl[card].money
                
        
    def buy_card(self, card):
        p = self.active_player
        if p.buys > 0:  
            if (self.supply_pile[card].values[0] > 0) & (p.money_in_play >= cl[card].cost): # can't buy if a card isn't there
                self.supply_pile[card] -= 1
                p.gain(card)
                p.money_in_play -= cl[card].cost
                p.buys -= 1
                
        if self.supply_pile["province"].values[0] == 0:
            self.end_game()
        
        if p.buys == 0:
            p.end_turn()
            self.next_turn()
            
    
    def next_turn(self):
        self.active_player_index = (self.active_player_index + 1) % self.num_players
        self.active_player = self.players[self.active_player_index]
        
    def end_game(self):
        for p in self.players:
            for card in p.all_cards:
                if cl[card].card_type == 'victory':
                    p.score += p.all_cards[card].values[0] * cl[card].victory_points
            print("Player %d : %d points" %(p.index, p.score))

In [439]:
g = GameState()
g.active_player.hand

,copper,silver,gold,estate,duchy,province
0,4,0,0,1,0,0


In [500]:
print(g.active_player.hand)
g.play_all_treasures()
print(g.active_player.money_in_play)

   copper  silver  gold  estate  duchy  province
0       2       2     0       0      1         0
6


In [501]:
g.buy_card("duchy")

In [502]:
print(g.players[0].discarded)
print(g.players[0].played)
print(g.players[0].hand)
print(g.players[0].buys)

   copper  silver  gold  estate  duchy  province
0       4       4     2       0      0         2
   copper  silver  gold  estate  duchy  province
0       0       0     0       0      0         0
   copper  silver  gold  estate  duchy  province
0       2       0     0       3      0         0
1


In [503]:
print(g.players[1].discarded)
print(g.players[1].played)
print(g.players[1].hand)
print(g.players[1].buys)

   copper  silver  gold  estate  duchy  province
0       5       4     1       0      2         0
   copper  silver  gold  estate  duchy  province
0       0       0     0       0      0         0
   copper  silver  gold  estate  duchy  province
0       1       1     0       3      0         0
1


In [505]:
g.end_game()

Player 0 : 15 points
Player 1 : 9 points


In [506]:
print(g.players[0].all_cards)
print(g.players[1].all_cards)

   copper  silver  gold  estate  duchy  province
0       7       4     2       3      0         2
   copper  silver  gold  estate  duchy  province
0       7       5     1       3      2         0
